## 加载数据集

In [1]:
import pandas as pd

train_labled_path = '../data/nCoV_100k_train.labled.csv'
test_path='../data/nCov_10k_test.csv'
df = pd.read_csv(train_labled_path, encoding='utf-8', usecols=[3,6]) # usecols读取指定列
df2 = pd.read_csv(test_path, encoding='utf-8', usecols=[0,3])
df = df[df['情感倾向'].isin(['0','-1','1'])]
print(df['情感倾向'].value_counts()) # 查看分数分布

0     57619
1     25392
-1    16902
Name: 情感倾向, dtype: int64


## 中文分词

In [2]:
import jieba # 导入分词包
df['微博中文内容']=df['微博中文内容'].map(str)
df['cuted']=df['微博中文内容'].map(lambda x:' '.join(jieba.cut(x)))
df2['微博中文内容']=df2['微博中文内容'].map(str)
df2['cuted']=df2['微博中文内容'].map(lambda x:' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\10369\AppData\Local\Temp\jieba.cache
Loading model cost 1.333 seconds.
Prefix dict has been built successfully.


In [3]:
print(df['cuted'][1])

开年 大 模型 … 累到 以为 自己 发烧 了 腰疼 膝盖 疼 腿疼 胳膊 疼 脖子 疼 # Luna 的 Krystallife # ?


## 训练集和测试集

In [4]:
# 输入和输出
X = df['cuted']
y = df['情感倾向']
x_ans = df2['cuted']

from sklearn.model_selection import train_test_split

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=0)
# sklearn机器学习库的train_test_split可以很方便进行划分数据集，一般默认25%做为测试集，但这里test_size表示百分之1的数据做测试集，百分之99的数据作为训练集，random_state指定随机种子保证每次划分的结果是一致的。

# 查看训练集
X_train.shape

(98913,)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer类可以实现文本的词袋表示，实例化对象后进行拟合，vocabulary_属性可以看到词表，每个单词对应的索引，不是频次。
# 变换器
vect = CountVectorizer()

vect.fit(X_train)

# 词表数量
print(len(vect.vocabulary_))
# 打印词表
print(vect.vocabulary_)

143120
{'湖北': 94670, '武汉': 88235, '重大项目': 132336, '投资': 72572, '2712': 2370, '亿元': 25145, '全市': 30876, '掀起': 74767, '项目': 138728, '建设': 64470, '热潮': 96665, '网页': 112235, '链接': 133601, '潜伏期': 95454, '这么': 128614, '进出': 128965, '地方': 49039, '检疫': 86872, '有用吗': 82491, '想出': 69293, '可以': 42569, '医学观察': 38035, '期再': 82891, '出去': 34299, '其实': 32144, '进不出': 128946, '集中': 136940, '治疗': 91465, '消毒': 93368, '隔离': 136639, '有效': 82372, '战疫': 70877, '打卡': 71641, '行动': 120359, '面对': 138190, '疫情': 101927, '承诺': 72364, '特殊': 98227, '时期': 79927, '不吃': 18065, '生食': 100831, '每天': 88688, '表达': 120788, '态度': 68006, '每个': 88651, '坚持': 49372, '意义': 69458, '活动': 92386, '期间': 82942, '新浪': 78349, '疫区': 101913, '捐助': 74529, '负压': 124549, '救护车': 76996, '孤单': 55991, '上海': 17373, '肺炎': 114617, '近期': 128327, '出现': 34417, '一例': 14875, '新型': 78200, '冠状病毒': 33045, '感染': 69635, '疑似': 101853, '患者': 68891, '小伙伴': 58990, '出门': 34507, '记得': 122707, '口罩': 42124, '少去': 60113, '人员': 24598, '密集': 58127, '场所': 49189, '注意': 91880

In [6]:
# words_matrix = pd.DataFrame(vect.transform(X).toarray(),columns=vect.get_feature_names())
#
# words_matrix.head()

## 构建模型

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 提取特征后，我们通过构建LogisticRegression逻辑回归分类器来拟合训练集数据，并使用交叉验证对LogisticRegression进行评估模型的性能。
# 交叉验证评估模型
scores = cross_val_score(LogisticRegression(),
                         vect.transform(X_train), y_train, cv=5)
print('平均交叉验证准确率：{:.3f}'.format(np.mean(scores)))

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

平均交叉验证准确率：0.710


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 去除停用词

In [8]:
# 我们会发现单词特征中包含很多数字及其他如“的”、“哦”等单词，这些单词在大多数情况下对于我们目前的案例没有提供有效的信息量，所以需要从原来的词袋中删除掉以提高模型的性能，对于这些特定词我们称之为停用词
def stopwords_list(d):
    with open('../data/'+d,'rb') as f:
        lines = f.readlines()
        result = [i.decode().strip('\n') for i in lines]
    return result
# 以下是引用之前所存储好的停用词表的文件
stopwords = stopwords_list('hit_stopwords.txt') # 标点符号
stopwords.extend(stopwords_list('cn_stopwords.txt'))
stopwords.extend(stopwords_list('baidu_stopwords.txt'))
stopwords.extend(stopwords_list('scu_stopwords.txt'))

In [9]:
# 重新构建单词矩阵
# max_df表示舍弃最频繁的单词，min_df表示每个词必须要在3个评论中出现，stop_words对于中文需要指定停用词的列表，最后使用正则表达式去掉所有数字
vect = CountVectorizer(max_df=0.8, min_df=3, stop_words=stopwords,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b')

vect.fit(X_train)

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'exp', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '若果', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


CountVectorizer(max_df=0.8, min_df=3,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃ ', '&', '*', '一一', '~~~~', '’', '. ', '『', '.一',
                            './', '-- ', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...],
                token_pattern='(?u)\\b[^\\d\\W]\\w+\\b')

In [10]:
print(vect.get_feature_names())

['__', '___', '____', '_____', '__________', 'a1', 'a2n', 'a3g389', 'a4', 'a50', 'a6g126', 'a6hzqlzo', 'a6pfl4v7', 'a6pfslok', 'a6pinnpb', 'a6piph89', 'a6pk09lq', 'a6pnneiy', 'a6px4ivp', 'a6pxmou3', 'a6pz9kzz', 'aa', 'aaa', 'ab', 'ababebaci', 'abc', 'abc2017', 'abcd', 'absolutecb', 'abyss', 'ac', 'acca', 'ace', 'ace2', 'acfc', 'ad', 'adam0616', 'aed', 'aeolus', 'aerosol', 'aesopbach', 'aesopbach75', 'afc', 'afrabot', 'agent', 'ai', 'aiba', 'aibofold', 'aids', 'aifxd1ng', 'aikkkkkkkkkkkkk', 'airpods', 'ait9zhe9', 'aj', 'ak', 'ak20190515', 'aka', 'akalui', 'akb48teamsh', 'aki', 'akira', 'akiramiya', 'alan', 'alan8616', 'alex', 'alexie', 'aling', 'allen', 'alliance', 'allvis', 'almighty', 'aloha', 'alter', 'am37', 'amazing', 'amortentiaz', 'amy', 'ana', 'anan', 'anaparastasi', 'androktasiai', 'ane', 'angel', 'angelababy', 'angeladoge', 'anita', 'ann606', 'anna', 'annnnnnn', 'anqi', 'ao', 'aoa', 'aoide', 'aokawa', 'ap', 'app', 'appo', 'appstore', 'aqi', 'ar', 'arashi', 'ariel', 'ariellecte

D:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
#words_matrix = pd.DataFrame(vect.transform(X_train).toarray(),columns=vect.get_feature_names())

In [12]:
# 训练模型
# 再次评估
lr=LogisticRegression()
lr.fit(vect.transform(X_train), y_train)

print('测试集准确率：{:.3f}'.format(lr.score(vect.transform(X_test), y_test)))

测试集准确率：0.683


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 用tf-idf缩放数据

In [13]:
# tf-idf 是一种用于资讯检索与文本挖掘的常用加权技术。tf-idf 是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

 # scikit-learn 在两个类中实现了 tf-idf 方法：TfidfTransformer 和 TfidfVectorizer， 前者接受 CountVectorizer 生成的稀疏矩阵并将其变换， 后者接受文本 数据并完成词袋特征提取与 tf-idf 变换。
pipe = make_pipeline(TfidfVectorizer(min_df=3), LogisticRegression())
pipe.fit(X_train, y_train)
scores = cross_val_score(pipe, X_train, y_train, cv=5)
# 再次评估准确率
print('平均交叉验证准确率：{:.3f}'.format(np.mean(scores)))

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

平均交叉验证准确率：0.731


In [14]:
# tf-idf较低的词要么在评论中经常出现，要么就是很少出现，tf-idf较大的词往往在评论中经常出现。
vectorizer = pipe.named_steps['tfidfvectorizer']
# 找到每个特征中最大值
max_value = vectorizer.transform(X_train).max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 获取特征名称
feature_names = np.array(vectorizer.get_feature_names())

print("tfidf较低的特征：\n{}".format(feature_names[sorted_by_tfidf[:20]]))
print()
print("tfidf较高的特征：\n{}".format( feature_names[sorted_by_tfidf[-20:]]))

tfidf较低的特征：
['祝迪丽' 'rakuen' 'ellis' '沙宣型' '唯粉霞霞' '饭心' '勉励' '模具' '芒果汁' '桃泥' '分满' '百丽'
 '智点' '巡店' '却表' '部省' '15813314017' '堪舆' '堪舆家' '掌握分寸']

tfidf较高的特征：
['难听' '歌唱' '很漂亮' '一天' '不要' '难搞' '安排' '安心' '影评' '脊梁' '彻底' '轻症' '鸡汤' '难过'
 '安全' '守望相助' '华晨' '祈福' '没人管' '赵赵']


D:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
from sklearn import metrics

# 预测值
y_pred = pipe.predict(X_test)

print('测试集准确率：{:.3f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('测试集准确率：{:.3f}'.format(pipe.score(X_test, y_test)))

metrics.confusion_matrix(y_test, y_pred)

测试集准确率：0.715
测试集准确率：0.715


array([[ 63,  92,   7],
       [ 37, 518,  42],
       [ 10,  97, 134]], dtype=int64)

In [16]:
print(y_pred)

['0' '0' '0' '0' '1' '0' '0' '-1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '-1' '1' '0' '0' '1' '0' '0' '-1'
 '0' '0' '-1' '1' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0'
 '0' '0' '0' '1' '0' '-1' '1' '0' '0' '0' '1' '-1' '0' '1' '0' '0' '0' '0'
 '0' '0' '0' '1' '0' '0' '0' '0' '-1' '0' '0' '1' '0' '0' '1' '0' '0' '0'
 '0' '0' '0' '1' '0' '-1' '0' '0' '0' '0' '-1' '0' '1' '-1' '0' '0' '0'
 '0' '0' '-1' '0' '0' '1' '-1' '0' '0' '0' '0' '0' '1' '1' '1' '0' '-1'
 '0' '0' '-1' '0' '0' '1' '0' '0' '1' '0' '0' '0' '-1' '1' '0' '0' '1' '0'
 '0' '-1' '0' '-1' '0' '-1' '1' '-1' '-1' '0' '0' '0' '0' '0' '0' '1' '1'
 '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '1' '0' '0' '0' '1' '0'
 '1' '1' '0' '0' '1' '0' '0' '1' '-1' '0' '0' '0' '0' '1' '0' '0' '1' '0'
 '0' '0' '0' '0' '-1' '0' '0' '1' '0' '1' '0' '1' '0' '1' '1' '-1' '0' '0'
 '1' '-1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0'
 '1' '0' '0' '1' '0' '-1' '1' '1' '0' '

In [17]:
print(pd.value_counts(y_pred))

0     707
1     183
-1    110
dtype: int64


## 输出

In [18]:
y_ans = pipe.predict(x_ans)

In [19]:
y_ans = pipe.predict(x_ans)
dict={"测试数据id":df2['微博id'].values.tolist(),'情感极性':y_ans.tolist()}
output_list = [df2['微博id'].values.tolist(),y_ans.tolist()]
output = pd.DataFrame(dict)
output.to_csv("../submit2.csv",sep=',',index=False)

In [20]:
print(output_list)

[[4456068992182160, 4456424178427250, 4456797466940200, 4456791021108920, 4457086404997440, 4457152129019640, 4457827361629760, 4458609708660190, 4458599680017930, 4458853061816280, 4459689523639890, 4459663745597810, 4458970833619120, 4460049088641370, 4460408020984770, 4460044734718810, 4460057116667200, 4460053245947360, 4460003047662520, 4460406158654420, 4460417973233560, 4460420603967690, 4460666649465060, 4460783620098110, 4461139519472380, 4461131160371150, 4461488888731490, 4462104544972710, 4462559836736440, 4462918689545870, 4463222482234740, 4463282205450510, 4468230968346510, 4468341035434870, 4468603934195360, 4463919710511440, 4464045383900520, 4464399178120380, 4464407126239860, 4464303207479990, 4470181806452450, 4470051229237290, 4470146385254120, 4464765973655970, 4465132082049670, 4465131238564520, 4465855921977370, 4455748258222440, 4466191457940130, 4466877365962200, 4467669631499950, 4467574181502060, 4468372098269660, 4460795306441440, 4460953988723200, 44691029